# Data Understanding

In [13]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import json
import os
from dotenv import load_dotenv

In [3]:
df = pd.read_csv('./DatafinitiElectronicsProductsPricingData.csv')

In [4]:
df.columns

Index(['id', 'prices.amountMax', 'prices.amountMin', 'prices.availability',
       'prices.condition', 'prices.currency', 'prices.dateSeen',
       'prices.isSale', 'prices.merchant', 'prices.shipping',
       'prices.sourceURLs', 'asins', 'brand', 'categories', 'dateAdded',
       'dateUpdated', 'ean', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'name', 'primaryCategories', 'sourceURLs', 'upc',
       'weight', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30'],
      dtype='object')

In [5]:
df[['dateAdded','dateUpdated']]

,dateAdded,dateUpdated
0,2015-04-13T12:00:51Z,2018-05-12T18:59:48Z
1,2015-05-18T14:14:56Z,2018-06-13T19:39:02Z
2,2015-05-18T14:14:56Z,2018-06-13T19:39:02Z
3,2015-05-18T14:14:56Z,2018-06-13T19:39:02Z
4,2015-05-18T14:14:56Z,2018-06-13T19:39:02Z
...,...,...
14587,2015-09-11T02:03:11Z,2018-06-13T19:43:55Z
14588,2015-09-11T02:03:11Z,2018-06-13T19:43:55Z
14589,2015-09-11T02:03:11Z,2018-06-13T19:43:55Z
14590,2015-09-11T02:03:11Z,2018-06-13T19:43:55Z


In [6]:
# to get datetime format
df['dateAdded'] = df['dateAdded'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
df['dateUpdated'] = df['dateUpdated'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

### Scraped unemployment rate from US Bureau of Labour Statistics Open API

In [7]:
# to get the unemployment rate in the US from US Bureau of Labour Statistics for a given date
# reference: https://www.bls.gov/developers/api_python.htm#python2
# reference: https://www.bls.gov/developers/home.htm
def get_unemployment_rate(date, cache):
    date_str = date.strftime('%Y-%m')
    if date_str in cache:
        return cache[date_str]
    # load .env and get api key
    load_dotenv()
    API_KEY = os.getenv('API_KEY')
    url = f'https://api.bls.gov/publicAPI/v2/timeseries/data/LNS14000000?startyear={date.year}&endyear={date.year}&registrationkey={API_KEY}'
    response = requests.get(url)
    data = response.json()
    
    if 'Results' in data:
        for item in data['Results']['series'][0]['data']:
            if item['periodName'] == date.strftime('%B') and item['year'] == str(date.year):
                rate = item['value']
                cache[date_str] = rate
                return rate
    return None

# Load cache if it exists
cache_file = 'unemployment_cache.json'
if os.path.exists(cache_file):
    with open(cache_file, 'r') as f:
        cache = json.load(f)
else:
    cache = {}

# Get the unemployment rate for the datetime objects
df['unemploymentRatedateAdded'] = df['dateAdded'].apply(lambda x: get_unemployment_rate(x, cache))
df['unemploymentRatedateUpdated'] = df['dateUpdated'].apply(lambda x: get_unemployment_rate(x, cache))

# Save cache to file
with open(cache_file, 'w') as f:
    json.dump(cache, f)

In [8]:
df[['unemploymentRatedateAdded','unemploymentRatedateUpdated']]

,unemploymentRatedateAdded,unemploymentRatedateUpdated
0,5.4,3.8
1,5.6,4.0
2,5.6,4.0
3,5.6,4.0
4,5.6,4.0
...,...,...
14587,5.0,4.0
14588,5.0,4.0
14589,5.0,4.0
14590,5.0,4.0


In [9]:
df

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,unemploymentRatedateAdded,unemploymentRatedateUpdated
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,...,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.93796E+11,32.8 pounds,NaN,NaN,NaN,NaN,NaN,5.4,3.8
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN,5.6,4.0
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN,5.6,4.0
3,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,NaN,...,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN,5.6,4.0
4,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,NaN,...,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN,5.6,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,AVpgibRDLJeJML43PTZX,65.99,65.99,NaN,NaN,USD,2015-09-05T00:00:00Z,True,NaN,USD 13.81 shipping,...,https://www.walmart.com/reviews/product/441840...,8.40005E+11,4.1 pounds,NaN,NaN,NaN,NaN,NaN,5.0,4.0
14588,AVpgibRDLJeJML43PTZX,58.99,58.99,Yes,New,USD,"2017-10-10T19:00:00Z,2017-09-06T17:00:00Z,2017...",False,Bestbuy.com,NaN,...,https://www.walmart.com/reviews/product/441840...,8.40005E+11,4.1 pounds,NaN,NaN,NaN,NaN,NaN,5.0,4.0
14589,AVpgibRDLJeJML43PTZX,58.49,58.49,In Stock,New,USD,2018-03-05T11:00:00Z,False,Walmart.com,Standard,...,https://www.walmart.com/reviews/product/441840...,8.40005E+11,4.1 pounds,NaN,NaN,NaN,NaN,NaN,5.0,4.0
14590,AVpgibRDLJeJML43PTZX,77.98,77.98,NaN,NaN,USD,2016-03-22T00:00:00Z,True,NaN,NaN,...,https://www.walmart.com/reviews/product/441840...,8.40005E+11,4.1 pounds,NaN,NaN,NaN,NaN,NaN,5.0,4.0


In [10]:
df.to_csv('after employment rates.csv')